In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definer variabler der bruges til at filtrere data med.

In [2]:
NB = pd.to_datetime('01-05-2023')#indflytningsdato for NordBlok NB md-dd-yyyy

MA = 10 #Maksimal alder i år ved indflytningen

NBMA = NB.replace(year=NB.year - MA)#dato for ælste udstyr der kan flyttes i NordBlokken

BG = 5000 #BagatelGrænse for pris

filtAfd = False #Skal der filterers på afdelinger

EJER = 'Bispebjerg og Frederiksberg Hospital'
#EJER = 'Nordsjællands Hospital'
#print(NBMA)
FilterInput = pd.DataFrame({'Filter funktion':['Ejer', 'Indflytnings dato', 'Udstyrets minimum alder [år]','Nyeste dato [dato]', 'Bagatelgrænse i kr', 'Ekskluderede grupper'],
                           'Værdi': [EJER, NB, MA, NBMA, BG, 'nan']})

# Læs data

In [3]:
filename = ['BBH.xlsx','FRH.xlsx'] #'BBH+FRH 19-03-20.xlsx' #filnavn på Meudusa udtræk i excel

filterIndput = 'filterIndput.xlsx' #Er en fil med input til filtrering, som er uafhængig af Medusa udtræk

AFD = 'Afdelinger.xlsx' #Hvis vi vil sortere afdelinger ud, kan gøres vha dette regneark.

df = pd.DataFrame()
for fil in filename:
    df = pd.read_excel(fil).append(df,sort=True) # Medusa udtræk
#df.info()
gz = pd.read_excel(filterIndput, sheet_name='Gråzone') # Gråzone udstyr

In [4]:
xmd = pd.read_excel(filterIndput, sheet_name='Xmodel') # Ekskluderete modeller

In [5]:
aFd = pd.read_excel(AFD)
#afd.head()

In [6]:
#Tilpasninger
df['Indkøbt']= pd.to_datetime(df.Indkøbt)#konverter til datetime som Pandas kan regne på

df["Pris"] = df[["Indkøbspris","Genansk.pris"]].max(axis=1)# find den største værdi og sæt det ind i ny kollonde "Pris"
#df.head()

In [7]:
#Det næste laver en liste over afdelinger og skriver det i et regneark - så vi kan vælge afdelinger ud som vi vil se nærmere på
if 0: #behøver kun at køre én gang - ænder 0 til 1 
    Afdl = pd.unique(df['Ejer (overafdeling) 2'])
    afdl = pd.DataFrame(Afdl)
    with pd.ExcelWriter('Afdelinger.xlsx') as writer:
        afdl.to_excel(writer,index = False)

# Så begynder filtereringen

In [8]:
x = df['Ejer (overafdeling) 2'].str.split(";", n = 1, expand = True)
df["Overafdeling"] =x[0]
df["Ejer"] = x[1]     

In [9]:
fdf = df.loc[df.Ejer.str.contains(EJER)== True] #filterer på Ejer
if filtAfd:
    fdf = fdf.loc[fdf.Overafdeling.isin(aFd['Afdeling'])] #filterer på afdeling
#fdf.head()

In [10]:
#fdf = fdf.loc[~fdf.Model.isin(xmd.Model),:] # fjern ekskluderede modeller
#ffdf = fdf.loc[fdf.Model.isin(xmd.Model),:] # find ekskluderede modeller
#print(len(ffdf))

In [11]:
fdf = fdf.loc[fdf.Indkøbt < NBMA,:] #filtrer på alder
#fdf.head()

In [12]:
fdf =fdf.loc[fdf.Pris > BG,:] #filtrer på pris
#fdf.head()

In [13]:
fdf = fdf.loc[~fdf.Gruppenavn.isin(gz.Gruppenavn)] #fjern gråzone udstyr

In [14]:
fdf = fdf.drop(['Indkøbspris', 'Genansk.pris', 'Ejer (overafdeling)'], axis=1)

In [15]:
#cols = fdf.columns.tolist()
#print(cols)

In [16]:
cols = ['App.nr', 'Gruppenavn', 'Fabrikat', 'Model', 'Indkøbt', 'Pris', 'Overafdeling', 'Afdeling (placering)', 'Afsnit (Placering)', 'Ejer']
fdf = fdf[cols]

In [17]:
fdf = fdf.sort_values(by=['Gruppenavn','Indkøbt'])#sorter på gruppenavn og indkøbsdato
#fdf.info()

# Opdeling i afdelinger

In [18]:
Afd = pd.unique(fdf['Overafdeling'])
Afd.sort()

#print(Afd)

fdf['Overafdeling'] = fdf['Overafdeling'].str.slice(0,27)
afd = pd.unique(fdf['Overafdeling'])
afd.sort()

i=0
antal =[]
pris =[]
for x in afd:
    xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
    antal.append(len(xls)) # find antal udstyr i afdelingen
    pris.append(xls['Pris'].sum()) # find den samlede værdi af udstyret 
    i = i+len(xls)
   # print(x[0:28], "  ",len(xls))    
#print(i) #opsummetet antal
#antal
StatusArk =pd.DataFrame(list(zip(Afd, antal, pris)), columns =['Afdeling', 'Antal udstyr', 'Værdi'])
#StatusArk

In [19]:
Grupper = pd.DataFrame(fdf.groupby(['Overafdeling', 'Gruppenavn'])['Gruppenavn'].count())
Grupper.unstack()

Gruppenavn                          \
Gruppenavn                  Afmagnetiseringsapparat Aftageapparat, låg/prop   
Overafdeling                                                                  
AMA., Akut modtageafdeling                      NaN                     NaN   
Abdominalcenter K                               NaN                     NaN   
Alpha-instituttet                               NaN                     NaN   
Anæstesi                                        NaN                     NaN   
Arbejds- og Miljømedicinsk                      NaN                     NaN   
Audiologisk ambulatorie, ud                     NaN                     NaN   
Centraloperationsgang, FRH                      NaN                     NaN   
Dermato-Venerologisk Afd. +                     NaN                     NaN   
Endokrinologisk Overafdelin                     NaN                     NaN   
FM-enheden                                      NaN                     NaN   
Fysio- og ergoterapiafdelin                     NaN                     NaN   
Geriatrisk afdeling G                           NaN                     NaN   
Geriatrisk og Palliativ ove                     NaN                     NaN   
Hygiejne Sygeplejersken                         NaN                     NaN   
IMT-afdeling                                    1.0                     NaN   
Institut for Sygdomsforebyg                     NaN                     NaN   
KVU - Kvalitet og Uddannels                     NaN                     NaN   
Kardiologisk afdeling Y                         NaN                     NaN   
Klinisk Farmakologisk Afdel                     NaN                     NaN   
Klinisk Fysiologisk Afdelin                     NaN                     NaN   
Klinisk biokemisk afdeling                      NaN                     1.0   
Koordinerende Forskningsenh                     NaN                     NaN   
Lungemedicinsk afdeling L                       NaN                     NaN   
Medicinsk og Geriatrisk Ove                     NaN                     NaN   
Neurologisk afd. N                              NaN                     NaN   
Ortopædkirurgisk afd. M                         NaN                     NaN   
Ortopædkirurgisk afdeling O                     NaN                     NaN   
Parkerinstituttet                               NaN                     NaN   
Patologiafdelingen                              NaN                     NaN   
Reumatologisk afdeling H, F                     NaN                     NaN   
Røntgenafdeling                                 NaN                     NaN   
Urologisk afdeling U, FRH                       NaN                     NaN   
Østerbroundersøgelsen                           NaN                     NaN   

                                                        \
Gruppenavn                  Analysator Analysator, DNA   
Overafdeling                                             
AMA., Akut modtageafdeling         NaN             NaN   
Abdominalcenter K                  NaN             NaN   
Alpha-instituttet                  NaN             NaN   
Anæstesi                           NaN             NaN   
Arbejds- og Miljømedicinsk         NaN             NaN   
Audiologisk ambulatorie, ud        NaN             NaN   
Centraloperationsgang, FRH         NaN             NaN   
Dermato-Venerologisk Afd. +        1.0             NaN   
Endokrinologisk Overafdelin        NaN             NaN   
FM-enheden                         NaN             NaN   
Fysio- og ergoterapiafdelin        NaN             NaN   
Geriatrisk afdeling G              NaN             NaN   
Geriatrisk og Palliativ ove        NaN             NaN   
Hygiejne Sygeplejersken            NaN             NaN   
IMT-afdeling                       NaN             NaN   
Institut for Sygdomsforebyg        NaN             NaN   
KVU - Kvalitet og Uddannels        NaN             NaN   
Kardiologisk afdeling Y            N

In [20]:
with pd.ExcelWriter('Udskiftningsbehov_Opdelt.xlsx') as writer:
    FilterInput.to_excel(writer, sheet_name='Opsumering',index = False)
    gz.to_excel(writer, sheet_name='Opsumering',startrow = 6,startcol = 1,index = False)
    StatusArk.to_excel(writer, sheet_name='Opsumering',startcol = 4, index = False)
    Grupper.to_excel(writer, sheet_name='Udstyrstyper i afdelingerne',index = True)
    Grupper.unstack().to_excel(writer, sheet_name='Udstyrstyper i afdelingerne', startcol = 5,index = True)
    
    for x in afd:
        xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
        if len(xls)>0:
            xls.to_excel(writer, sheet_name=x[0:27],index = False)

# Skriv det hele i et regneark

In [21]:
with pd.ExcelWriter('Udskiftningsbehov.xlsx') as writer:
    
    fdf.to_excel(writer,index = False)